**Author:** Lisa Wallner  
**Description:**  
 

*Hint: If lines are created with support of a Large Language Model or the code is taken from another source, you find following hint at the end of the line: (generated with Microsoft Copilot) or (source: link_to_source)*

In [116]:
import pandas as pd
import json
import matplotlib as plt
import numpy as np
from scipy.stats import mode

## Dataframe df_llama  
This dataframe gives an overview of the evaluation results from the original and genearted READMES (generated by llama3.1-8b). The scoring was created with three different kind of llms - llama3.1-8b, reka-flash & jamba-1.5-mini.

### Universal Functions

In [117]:
def exclude_repos(df):
    repos_exclude = [('diego-vicente', 'som-tsp'), ('Taniiishk', 'Rock-Paper-Scissors-Game')]
    df = df[~df.set_index(['repo_owner', 'repo_name']).index.isin(repos_exclude)] 

    return df

In [118]:
def remove_cols(df):
    df = df.drop(['readme_g_score', 'readme_g_score_q1_txt', 'readme_g_score_q2_txt', 'readme_g_score_q3_txt', 'readme_g_score_q4_txt', 'readme_g_score_q5_txt', 'readme_o_score', 'readme_o_score_q1_txt', 'readme_o_score_q2_txt', 'readme_o_score_q3_txt', 'readme_o_score_q4_txt', 'readme_o_score_q5_txt'], axis=1)

    return df

In [119]:
def load_data(path):
    with open(path, 'r') as f:
        loaded_data = json.load(f)

    df = pd.DataFrame(data=loaded_data)

    return df

### Model 1

In [120]:
model = 'model1'
path = f'../data/df_score_lama/df_score_{model}_manuel.json'

In [121]:
df_1 = load_data(path)

During the process there are some repositories found which can not be used because of inconsistence in the data. These repository data will be filterd out.  
+ diego-vicente: som-tsp --> GitHub repository has no README file
+ Taniiishk: Rock-Paper-Scissors-Game --> GitHub repository has no license

In [122]:
len(df_1)

201

In [123]:
df_1.head(2)

,repo_owner,repo_name,readme_g_score,readme_g_score_q1,readme_g_score_q1_txt,readme_g_score_q2,readme_g_score_q2_txt,readme_g_score_q3,readme_g_score_q3_txt,readme_g_score_q4,...,readme_o_score_q1,readme_o_score_q1_txt,readme_o_score_q2,readme_o_score_q2_txt,readme_o_score_q3,readme_o_score_q3_txt,readme_o_score_q4,readme_o_score_q4_txt,readme_o_score_q5,readme_o_score_q5_txt
0,keras-rl,keras-rl,"### ""q1"": [\n ##""score"": 5##,\n ##""expla...",5,The README clearly states that the project is ...,5,The README clearly explains that the project i...,4,The README provides a clear example code to ge...,2,...,5,The goal of the project is clearly stated in t...,5,The project is useful because it provides a se...,4,The README provides a clear installation guide...,5,The README provides several ways for users to ...,5,The README clearly states that the project is ...
1,terraform-compliance,cli,"### ""q1"": [ \n ##""score"": 5##,\n ##""expl...",5,"The project's goal is clearly stated in the ""D...",5,The project is useful because it provides a BD...,4,The README provides a clear step-by-step guide...,2,...,5,The goal of the project is clearly stated in t...,5,The README clearly explains why the project is...,4,The README provides a clear installation guide...,5,The README provides several ways for users to ...,5,The README clearly states that the project is ...


In [124]:
df_1 = exclude_repos(df_1)

In [125]:
len(df_1)

199

For the final dataframe not all columns are required. Dispensable columns are removed to keep the dataframe clean.

In [126]:
df_1 = remove_cols(df_1)

In [127]:
df_1.describe()

,readme_g_score_q1,readme_g_score_q2,readme_g_score_q3,readme_g_score_q4,readme_g_score_q5,readme_o_score_q1,readme_o_score_q2,readme_o_score_q3,readme_o_score_q4,readme_o_score_q5
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,4.542714,4.577889,4.316583,2.135678,2.266332,4.693467,4.743719,4.296482,4.221106,4.311558
std,0.686769,0.740402,0.581948,0.701142,0.950503,0.798678,0.703274,0.743686,1.133225,1.264678
min,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000,2.000000,2.000000,5.000000,5.000000,4.000000,4.000000,4.000000
50%,5.000000,5.000000,4.000000,2.000000,2.000000,5.000000,5.000000,4.000000,5.000000,5.000000
75%,5.000000,5.000000,5.000000,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Model 2

In [128]:
model = 'model2'
path = f'../data/df_score_lama/df_score_{model}_manuel.json'

In [129]:
df_2 = load_data(path)

In [130]:
len(df_2)

201

In [131]:
df_2.head(2)

,repo_owner,repo_name,readme_g_score,readme_g_score_q1,readme_g_score_q1_txt,readme_g_score_q2,readme_g_score_q2_txt,readme_g_score_q3,readme_g_score_q3_txt,readme_g_score_q4,...,readme_o_score_q1,readme_o_score_q1_txt,readme_o_score_q2,readme_o_score_q2_txt,readme_o_score_q3,readme_o_score_q3_txt,readme_o_score_q4,readme_o_score_q4_txt,readme_o_score_q5,readme_o_score_q5_txt
0,keras-rl,keras-rl,"###\n""q1"": [\n ""score"": 4,\n ""explanati...",4,The README clearly states the goal of the proj...,3,The usefulness of the project is hinted at by ...,4,The README offers a clear guide on how to get ...,2,...,5,The README clearly states the goal of the proj...,4,The project is useful because it provides a st...,5,Users can get started with the project very ea...,4,Users can get help through several channels: f...,4,The README mentions that the project is mainta...
1,terraform-compliance,cli,"###\n""q1"": [\n ""score"": 4,\n ""explanati...",4,The README clearly states that the project is ...,3,The usefulness of the project is implied by it...,5,The README provides clear and detailed instruc...,2,...,5,The README clearly states the goal of the proj...,5,The project's usefulness is well-explained thr...,4,Users can get started with the project by foll...,3,Help resources are mentioned through links to ...,3,The README mentions the project maintainer (ee...


In [132]:
df_2 = exclude_repos(df_2)

In [133]:
len(df_2)

199

In [134]:
df_2 = remove_cols(df_2)

In [135]:
df_2.describe()

,readme_g_score_q1,readme_g_score_q2,readme_g_score_q3,readme_g_score_q4,readme_g_score_q5,readme_o_score_q1,readme_o_score_q2,readme_o_score_q3,readme_o_score_q4,readme_o_score_q5
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,4.346734,3.432161,4.663317,2.331658,3.211055,4.758794,4.537688,4.693467,4.020101,4.206030
std,0.546224,0.545108,0.514645,0.513015,0.508194,0.561483,0.701612,0.620775,0.893070,0.786736
min,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000
25%,4.000000,3.000000,4.000000,2.000000,3.000000,5.000000,4.000000,5.000000,4.000000,4.000000
50%,4.000000,3.000000,5.000000,2.000000,3.000000,5.000000,5.000000,5.000000,4.000000,4.000000
75%,5.000000,4.000000,5.000000,3.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Model 3

In [136]:
model = 'model3'
path = f'../data/df_score_lama/df_score_{model}_manuel.json'

In [137]:
df_3 = load_data(path)

In [138]:
len(df_3)

201

In [139]:
df_3.head(2)

,repo_owner,repo_name,readme_g_score,readme_g_score_q1,readme_g_score_q1_txt,readme_g_score_q2,readme_g_score_q2_txt,readme_g_score_q3,readme_g_score_q3_txt,readme_g_score_q4,...,readme_o_score_q1,readme_o_score_q1_txt,readme_o_score_q2,readme_o_score_q2_txt,readme_o_score_q3,readme_o_score_q3_txt,readme_o_score_q4,readme_o_score_q4_txt,readme_o_score_q5,readme_o_score_q5_txt
0,keras-rl,keras-rl,"###\n""q1"": [\n ""score"": 4,\n ""explanati...",4,The goal of the project is clearly stated in t...,5,The project's usefulness is well-explained in ...,3,"While the installation instructions are clear,...",3,...,5,The goal of the project is clearly stated in t...,5,The project is useful because it provides a li...,5,The README provides clear instructions on how ...,5,The README provides information on where users...,5,The README provides information on who maintai...
1,terraform-compliance,cli,"###\n""q1"": [\n ""score"": 5,\n ""explanati...",5,The goal of the project is clearly stated in t...,5,The project's usefulness is evident from its d...,4,The installation instructions are clear and co...,3,...,5,The goal of the project is clearly stated in t...,5,The project's usefulness is well explained. It...,5,"Getting started is explained in the ""Installat...",5,Help is readily available through the provided...,5,The project's maintenance and contribution det...


In [140]:
df_3 = exclude_repos(df_3)

In [141]:
len(df_3)

199

In [142]:
df_3 = remove_cols(df_3)

In [143]:
df_3.describe()

,readme_g_score_q1,readme_g_score_q2,readme_g_score_q3,readme_g_score_q4,readme_g_score_q5,readme_o_score_q1,readme_o_score_q2,readme_o_score_q3,readme_o_score_q4,readme_o_score_q5
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,4.793970,4.834171,4.361809,3.698492,4.241206,4.929648,4.849246,4.552764,4.281407,4.633166
std,0.474356,0.479518,0.710490,1.123463,0.949274,0.369378,0.468597,0.624403,0.969852,0.766143
min,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,5.000000,5.000000,4.000000,3.000000,3.000000,5.000000,5.000000,4.000000,3.000000,5.000000
50%,5.000000,5.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Final Dataframe - df_llama

Create Dataframe for the scoring data of 199 GitHub repositories and the three evaluation models lama3.1-8b, reka-flash and jamba-1.5-mini.

In [144]:
cols = ['repo_owner', 'repo_name', 'model_type', 'total_score_g', 'score_q1_g', 'score_q2_g', 'score_q3_g', 'score_q4_g', 'score_q5_g', 'total_score_o', 'score_q1_o', 'score_q2_o', 'score_q3_o', 'score_q4_o', 'score_q5_o']
df_llama = pd.DataFrame(columns=cols)

In [145]:
dfs = [df_1, df_2, df_3]
model_types = ['llama3.1-8b', 'reka-flash', 'jamba-1.5-mini']

In [146]:
for idx, df in enumerate(dfs):
    model_type = model_types[idx]
    
    for _,row in df.iterrows():
        new_row = {
            'repo_owner': row['repo_owner'], 
            'repo_name': row['repo_name'], 
            'model_type': model_type, 
            'total_score_g': np.nan, 
            'score_q1_g': row['readme_g_score_q1'], 
            'score_q2_g': row['readme_g_score_q2'], 
            'score_q3_g': row['readme_g_score_q3'], 
            'score_q4_g': row['readme_g_score_q4'], 
            'score_q5_g': row['readme_g_score_q5'], 
            'total_score_o': np.nan, 
            'score_q1_o': row['readme_o_score_q1'], 
            'score_q2_o': row['readme_o_score_q2'], 
            'score_q3_o': row['readme_o_score_q3'], 
            'score_q4_o': row['readme_o_score_q4'], 
            'score_q5_o': row['readme_o_score_q5']
    }
        df_llama = pd.concat([df_llama, pd.DataFrame([new_row])], ignore_index=True)

Calculate the total score for the genearted and orginal README's.

In [147]:
df_llama['total_score_g'] = df_llama[['score_q1_g', 'score_q2_g', 'score_q3_g', 'score_q4_g', 'score_q5_g']].sum(axis=1).astype(int)
df_llama['total_score_o'] = df_llama[['score_q1_o', 'score_q2_o', 'score_q3_o', 'score_q4_o', 'score_q5_o']].sum(axis=1).astype(int)

In [148]:
df_llama

,repo_owner,repo_name,model_type,total_score_g,score_q1_g,score_q2_g,score_q3_g,score_q4_g,score_q5_g,total_score_o,score_q1_o,score_q2_o,score_q3_o,score_q4_o,score_q5_o
0,keras-rl,keras-rl,llama3.1-8b,18,5,5,4,2,2,24,5,5,4,5,5
1,terraform-compliance,cli,llama3.1-8b,18,5,5,4,2,2,24,5,5,4,5,5
2,openstack,openstack,llama3.1-8b,18,5,4,5,2,2,20,4,5,2,5,4
3,OlafenwaMoses,ImageAI,llama3.1-8b,18,5,4,5,2,2,23,4,5,4,5,5
4,Bitwise-01,Instagram-,llama3.1-8b,18,5,4,5,2,2,19,4,4,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,Shubhamsaboo,awesome-llm-apps,jamba-1.5-mini,19,4,4,5,3,3,25,5,5,5,5,5
593,databrickslabs,dolly,jamba-1.5-mini,20,5,5,4,3,3,21,5,4,5,3,4
594,Genesis-Embodied-AI,Genesis,jamba-1.5-mini,22,5,5,4,3,5,24,5,5,4,5,5
595,Anjok07,ultimatevocalremovergui,jamba-1.5-mini,25,5,5,5,5,5,22,5,5,4,3,5


In [149]:
df_llama.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   repo_owner     597 non-null    object
 1   repo_name      597 non-null    object
 2   model_type     597 non-null    object
 3   total_score_g  597 non-null    int64 
 4   score_q1_g     597 non-null    object
 5   score_q2_g     597 non-null    object
 6   score_q3_g     597 non-null    object
 7   score_q4_g     597 non-null    object
 8   score_q5_g     597 non-null    object
 9   total_score_o  597 non-null    int64 
 10  score_q1_o     597 non-null    object
 11  score_q2_o     597 non-null    object
 12  score_q3_o     597 non-null    object
 13  score_q4_o     597 non-null    object
 14  score_q5_o     597 non-null    object
dtypes: int64(2), object(13)
memory usage: 70.1+ KB


In [150]:
# df_llama['total_score_o'][df_llama['model_type'] == 'llama3.1-8b'].value_counts()

## Dataframe df_llama_com  
This dataframe contains the compressed results from df_llama.

### README's with summaries and subsummaries

In [151]:
df_llama_tmp = df_llama[['repo_owner', 'repo_name', 'total_score_g', 'total_score_o', 'model_type']] # 'score_g', 'score_o', 'subsummary', 'subsummary_cnt']]

Add column with ids.

In [152]:
df_llama_tmp['id'] = df_llama_tmp['repo_owner'] + '_' + df_llama_tmp['repo_name']

/tmp/ipykernel_94669/1442397247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_llama_tmp['id'] = df_llama_tmp['repo_owner'] + '_' + df_llama_tmp['repo_name']


In [153]:
df_llama_tmp[df_llama_tmp['repo_name'] == 'keras-rl']
# 'keras-rl'

,repo_owner,repo_name,total_score_g,total_score_o,model_type,id
0,keras-rl,keras-rl,18,24,llama3.1-8b,keras-rl_keras-rl
199,keras-rl,keras-rl,16,22,reka-flash,keras-rl_keras-rl
398,keras-rl,keras-rl,18,25,jamba-1.5-mini,keras-rl_keras-rl


In [154]:
df_llama_tmp

,repo_owner,repo_name,total_score_g,total_score_o,model_type,id
0,keras-rl,keras-rl,18,24,llama3.1-8b,keras-rl_keras-rl
1,terraform-compliance,cli,18,24,llama3.1-8b,terraform-compliance_cli
2,openstack,openstack,18,20,llama3.1-8b,openstack_openstack
3,OlafenwaMoses,ImageAI,18,23,llama3.1-8b,OlafenwaMoses_ImageAI
4,Bitwise-01,Instagram-,18,19,llama3.1-8b,Bitwise-01_Instagram-
...,...,...,...,...,...,...
592,Shubhamsaboo,awesome-llm-apps,19,25,jamba-1.5-mini,Shubhamsaboo_awesome-llm-apps
593,databrickslabs,dolly,20,21,jamba-1.5-mini,databrickslabs_dolly
594,Genesis-Embodied-AI,Genesis,22,24,jamba-1.5-mini,Genesis-Embodied-AI_Genesis
595,Anjok07,ultimatevocalremovergui,25,22,jamba-1.5-mini,Anjok07_ultimatevocalremovergui


Calculate mode or mean of score values.

In [155]:
# rows are grouped by 'id'
# then an aggregation mode is applied on the columns 'total_score_g' & 'total_score_o'
# with .iloc[0] the first value of x.mode() is taken (mode() can return multiple results) (if 2/3 llms gave the same score, take this)
# if there are only unique values in x.mode() the mean value should be calcualted instead
# len(x.unique()) == 3 checks if x.unique() contains 3 different values, if yes, mean() will called
    # s = pd.Series([19, 18, 1])
    # s_u = s.unique()
    # print(s_u)
    # if len(s_u) == 0:
    #     print('empty')
# .reset_index() --> prepare index of current df for merging into another df
# .rename() --> customise name of column which holds the results of the aggregation
df_tmp_g = df_llama_tmp.groupby('id')['total_score_g'].agg(lambda x: x.mode().iloc[0] if len(x.unique()) != 3 else x.mean()).reset_index().rename(columns={'total_score_g': 'score_g'})
df_tmp_o = df_llama_tmp.groupby('id')['total_score_o'].agg(lambda x: x.mode().iloc[0] if len(x.unique()) != 3 else x.mean()).reset_index().rename(columns={'total_score_o': 'score_o'})

# merge df_tmp_o & df_tmp_g by key 'id' to df_llama_tmp
df_llama_tmp = df_llama_tmp.merge(df_tmp_g, on='id', how='left') \
                           .merge(df_tmp_o, on='id', how='left')

Remove duplicate rows.

In [156]:
df_llama_tmp = df_llama_tmp[~df_llama_tmp['model_type'].isin(['reka-flash', 'jamba-1.5-mini'])]

In [183]:
df_llama_com = df_llama_tmp[['id','repo_owner', 'repo_name', 'score_g', 'score_o']]

In [184]:
df_llama_com['subsummary'] = 0
df_llama_com['summary_cnt'] = 0

/tmp/ipykernel_94669/2102315245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_llama_com['subsummary'] = 0
/tmp/ipykernel_94669/2102315245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_llama_com['summary_cnt'] = 0


In [186]:
df_llama_com

,id,repo_owner,repo_name,score_g,score_o,subsummary,summary_cnt
0,keras-rl_keras-rl,keras-rl,keras-rl,18.000000,23.666667,0,1
1,terraform-compliance_cli,terraform-compliance,cli,19.000000,23.000000,0,1
2,openstack_openstack,openstack,openstack,20.000000,20.000000,0,1
3,OlafenwaMoses_ImageAI,OlafenwaMoses,ImageAI,17.000000,23.000000,1,5
4,Bitwise-01_Instagram-,Bitwise-01,Instagram-,18.666667,14.000000,0,1
...,...,...,...,...,...,...,...
194,Shubhamsaboo_awesome-llm-apps,Shubhamsaboo,awesome-llm-apps,17.000000,24.000000,1,5
195,databrickslabs_dolly,databrickslabs,dolly,17.000000,21.000000,0,1
196,Genesis-Embodied-AI_Genesis,Genesis-Embodied-AI,Genesis,17.666667,24.000000,1,16
197,Anjok07_ultimatevocalremovergui,Anjok07,ultimatevocalremovergui,20.333333,24.000000,1,5


Get information about summaries.

In [185]:
for idx, row in df_llama_com.iterrows():
    id = row['id']
    path = f'../data/output_readme_data_lama/{id}_output.json'

    with open(path, 'r') as f:
        loaded_data = json.load(f)

    subsummaries_cnt = len(loaded_data['summaries'])

    df_llama_com.loc[idx, 'subsummary'] = 1 if subsummaries_cnt > 1 else 0
    df_llama_com.loc[idx, 'summary_cnt'] = subsummaries_cnt

In [187]:
df_llama_com.describe()

,score_g,score_o,subsummary,summary_cnt
count,199.000000,199.000000,199.000000,199.00000
mean,19.137353,22.594640,0.351759,4.80402
std,1.845948,2.409962,0.478723,6.88042
min,9.333333,12.000000,0.000000,1.00000
25%,18.000000,21.666667,0.000000,1.00000
50%,19.333333,23.666667,0.000000,1.00000
75%,20.333333,24.000000,1.000000,6.50000
max,24.000000,25.000000,1.000000,46.00000


## Dataframe df_llama_o  
This dataframe contains the scoring for the original README files with addtional information about the GitHub repository.

In [188]:
df_llama_o = df_llama_com[['id', 'repo_owner', 'repo_name', 'score_o']]

In [190]:
df_meta = load_data(path='../data/df_repos_sample_250.json')

In [191]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 250 non-null    int64  
 1   name               250 non-null    object 
 2   full_name          250 non-null    object 
 3   html_url           250 non-null    object 
 4   description        250 non-null    object 
 5   url                250 non-null    object 
 6   labels_url         250 non-null    object 
 7   created_at         250 non-null    object 
 8   updated_at         250 non-null    object 
 9   pushed_at          250 non-null    object 
 10  size               250 non-null    int64  
 11  stargazers_count   250 non-null    int64  
 12  watchers_count     250 non-null    int64  
 13  language           250 non-null    object 
 14  has_issues         250 non-null    bool   
 15  has_projects       250 non-null    bool   
 16  has_downloads      250 non